In [18]:
# Set Up:
import sys
USING_COLAB = 'google.colab' in sys.modules

if USING_COLAB:
    !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
    !pip install -U renderlab
    !pip install -U colabgymrender
    !pip install -U moviepy==0.2.3.5
    !pip install imageio==2.4.1
    !pip install --upgrade AutoROM
    !AutoROM --accept-license
    !pip install stable_baselines3  
    !pip install "gymnasium[atari, accept-rom-licesnse]"

import numpy as np
import ale_py
import shimmy
import gymnasium as gym
import renderlab as rl
import random
import matplotlib.pyplot as plt
from colabgymrender.recorder import Recorder
from copy import deepcopy

from torch.utils.data import DataLoader
from torch import nn
import torch

from tqdm import tqdm, trange

seed = 24
data_seed = 700



In [19]:
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env.base_vec_env import VecEnv
from stable_baselines3.common.env_checker import check_env # for regular envs
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.vec_env import VecFrameStack
# not using make_atari_env bc that wraps it in an atari wrapper only for v4 stuff (needs verification)

In [20]:

print('gym:', gym.__version__)
print('ale_py:', ale_py.__version__)
print("GPU is available:", torch.cuda.is_available())

gym: 0.29.1
ale_py: 0.8.1
GPU is available: True


In [21]:
print(torch.version.cuda)
print(torch.__version__)

11.8
2.2.2


# Functions from A5 starter code

In [22]:
  # Setting the seed to ensure reproducability
def reseed(seed, env=None):
    '''
        Sets the seed for reproducibility

        When @param env is provided, also sets the
        random number generataor of the gym environment
        to this particular seed
    '''
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    if env is not None:
        env.unwrapped._np_random = gym.utils.seeding.np_random(seed)[0]

reseed(seed)

In [23]:
def visualize(env_name='ALE/Pacman-v5', algorithm=None, video_name="test", env_args={}):
    """Visualize a policy network for a given algorithm on a single episode

        Args:
            env_name: Name of the gym environment to roll out `algorithm` in, it will be instantiated using gym.make or make_vec_env
            algorithm (PPOActor): Actor whose policy network will be rolled out for the episode. If
            no algorithm is passed in, a random policy will be visualized.
            video_name (str): Name for the mp4 file of the episode that will be saved (omit .mp4). Only used
            when running on local machine.
    """

    def get_action(obs):
        if not algorithm:
            return env.action_space.sample()
        else:
            return algorithm.predict(obs)[0]

    if USING_COLAB:
        from renderlab import RenderFrame
        # visualization probably wont work on colab with my new changes

        directory = './video'
        env_args['render_mode'] = 'rgb_array'
        env = gym.make(env_name, **env_args)
        env = RenderFrame(env, directory)
        obs, info = env.reset()

        while True:
            action = get_action(obs)
            obs, reward, done, truncate, info = env.step(action)

            if done:
                break

        env.play()
    else:
        import cv2
        # need to specify dimensions correctly (use the print statement)
        video = cv2.VideoWriter(f"videos/{video_name}.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 24, (160,250))

        env_args['render_mode'] = 'rgb_array'
        #env = gym.make(env_name, **env_args)
        #env = gym.wrappers.AtariPreprocessing(env)
        env = make_atari_env(env_name, n_envs=1, env_kwargs=env_args)
        env = VecFrameStack(env, n_stack=4)
        #env.metadata['render_fps'] = 60
        obs = env.reset()
        #img = plt.imshow(env.render()) # only call this once
        while True:
            #img.set_data(env.render()) # just update the data
            #display.display(plt.gcf())
            #display.clear_output(wait=True)
            action = get_action(obs)
            if(not algorithm):    
                action = [action]
            obs, reward, done, info  = env.step(action)
            if done:
                print(reward)
                break
            im = env.render()
            im = im[:,:,::-1]
            #print(im.shape)

            video.write(im)

        video.release()
        env.close()
        print(f"Video saved as {video_name}.mp4")

In [24]:
def evaluate_policy(actor, environment, num_episodes=100, progress=True):
    '''
        Returns the mean trajectory reward of rolling out `actor` on `environment

        Parameters
        - actor: PPOActor instance, defined in Part 1
        - environment: classstable_baselines3.common.vec_env.VecEnv instance
        - num_episodes: total number of trajectories to collect and average over
    '''
    total_rew = 0

    iterate = (trange(num_episodes) if progress else range(num_episodes))
    for _ in iterate:
        obs = environment.reset()
        done = False

        while not done:
            action = actor.predict(obs)[0]

            next_obs, reward, done, info = environment.step(action)
            total_rew += reward

            obs = next_obs

    return (total_rew / num_episodes).item()


In [ ]:
# for testing if visualization works (random actions selected)
# env arguments are space invader specific
# see https://ale.farama.org/environments/#2 for description of params
# we use frameskip for training but for visualization we don't use frameskip so we can see what's happening clearly
# for dqn we evaluate on same environment as testing
env_id = "ALE/Pacman-v5"
env_args = {
    'mode':0,
    'difficulty':0,
    'obs_type':"rgb",
    'full_action_space':False,
    'frameskip':1
    }
visualize(env_id, env_args=env_args, video_name="pacman-test")
# reward is 0 because randomly picking actions gives you no right answers

[0.]
Video saved as pacman-test.mp4


# Initialization
from A5 starter code

In [9]:
# just one environment for evaluation
env1 = make_atari_env(env_id, n_envs=1, env_kwargs=env_args)
env1 = VecFrameStack(env1, n_stack=4)

In [10]:
# 8 environments
num_envs = 8
env8 = make_atari_env(env_id, n_envs=num_envs, env_kwargs=env_args)
env8 = VecFrameStack(env8, n_stack=4)

In [12]:
eval_callback = EvalCallback(
    eval_env=env1,
    callback_on_new_best=None,
    callback_after_eval=None,
    n_eval_episodes=20,
    eval_freq=max(200_000 // num_envs, 1),
    log_path='./logs/eval_logs_pacman_env1/',
    best_model_save_path='./models/best_ppo_model_env1_pacman/',
    deterministic=True,
    render=False,
    verbose=1,
    warn=True
)

# PPO Model

In [13]:
ppomodel = PPO(
    "CnnPolicy",
    env1,
    learning_rate=2.5e-4,
    n_steps=128,
    batch_size=256,
    n_epochs=4,
    gamma=0.99,
    gae_lambda=0.98,
    clip_range=0.1,
    ent_coef=0.01,
    vf_coef=0.5,
    max_grad_norm=0.5,
)

We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=128 and n_envs=1)
  warnings.warn(



## Training PPO model

In [14]:
reseed(seed)
# If you get the error that Only one live display may be active at once, either restart colab or set progress bar = False
total_timesteps = 10_000_000
ppomodel.learn(total_timesteps=total_timesteps, callback=eval_callback, progress_bar=True)

  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")



Output()

Eval num_timesteps=25000, episode_reward=1.45 +/- 1.96

Episode length: 1684.80 +/- 29.64

New best mean reward!

...

Episode length: 4237.85 +/- 486.70

Eval num_timesteps=9975000, episode_reward=255.85 +/- 137.35

Episode length: 4251.15 +/- 506.06

Eval num_timesteps=10000000, episode_reward=175.50 +/- 48.66

Episode length: 4103.15 +/- 359.86

In [15]:
best_ppo_model = PPO.load("models/best_ppo_model_env1_pacman/best_model.zip", env=env1)

print(evaluate_policy(best_ppo_model, env1))

100%|██████████| 100/100 [01:19<00:00,  1.26it/s]

20.719999313354492


In [16]:
visualize('ALE/Pacman-v5', algorithm=best_ppo_model, video_name='best_ppo_env1_pacman_video', env_args=env_args)

# might need to rerun with best_ppo_model

  logger.warn(



[0.]
Video saved as best_ppo_env1_pacman_video.mp4


In [17]:
ppomodel.save("models/pacman_ppo_10m_env1")

# best model_1 and best_model_2 are using difficulty 1

# DQN Model

Needs to train on a different env with different params due to dqn buffer memory size

In [27]:
from stable_baselines3 import DQN

# can only use env1 instead of env8 for training due to memory constraints

dqnmodel = DQN(
    "CnnPolicy",  # Using CNN policy for visual inputs
    env1,
    learning_rate=1e-4,
    buffer_size=100_000,            # Smaller replay buffer
    learning_starts=50_000,        # Start learning earlier
    batch_size=32,
    train_freq=4,                  # Train more frequently
    target_update_interval=1000,  # Update target network more frequently
    exploration_fraction=0.1,     # Faster exploration decay
    exploration_final_eps=0.01,
    gamma=0.99,
)

In [28]:
eval_callback_dqn = EvalCallback(
    eval_env=env1,
    callback_on_new_best=None,
    callback_after_eval=None,
    n_eval_episodes=10,
    eval_freq=200_000,
    log_path='./logs/pacman_dqn/',
    best_model_save_path='./models/pacman_dqn/',
    deterministic=True,
    render=False,
    verbose=1,
    warn=True
)

In [29]:
reseed(seed)

# If you get the error that Only one live display may be active at once, either restart colab or set progress bar = False
total_timesteps = 10_000_000
dqnmodel.learn(total_timesteps=total_timesteps, callback=eval_callback_dqn, progress_bar=True)

  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")



Output()

Eval num_timesteps=200000, episode_reward=29.90 +/- 11.69

Episode length: 2239.00 +/- 487.13

New best mean reward!

...

Episode length: 3860.60 +/- 276.64

Eval num_timesteps=9800000, episode_reward=181.00 +/- 63.49

Episode length: 4074.30 +/- 419.92

Eval num_timesteps=10000000, episode_reward=146.90 +/- 21.23

Episode length: 4117.40 +/- 240.12

In [30]:

best_dqn_model = DQN.load("models/pacman_dqn/best_model.zip", env=env1)
print(evaluate_policy(best_dqn_model, env1))

  warnings.warn(

100%|██████████| 100/100 [00:46<00:00,  2.15it/s]

27.200000762939453


In [31]:
visualize('ALE/Pacman-v5', algorithm=best_dqn_model, video_name='dqn_pacman', env_args=env_args)

[0.]
Video saved as dqn_pacman.mp4
